In [12]:
from algorithms import mlsl, dual_annealing, cmaes
from framework import SaveHandler, MAX_INSTANCE_FES
from pprint import pprint

## MLSL
### Information and performance measures for the best instance selected

In [13]:
save_handler = SaveHandler('mlsl')
algo = mlsl
algo.load_best_instance(save_handler=save_handler)
ins = algo.best_instance
ins.run(save_handler=save_handler, restart=True, max_instance_fes=MAX_INSTANCE_FES * 5)
pprint(ins.info)
pprint(ins.params)
pprint(ins.performance_measures(excluding_first=True))
pprint(len(ins.results))
pprint(sum([r.eval_num for r in ins.results]))

{'algorithm_name': 'MLSL', 'algorithm_version': 1, 'instance_index': 960}
{'population': 961}
{'avg_height': 1316.6317561195845,
 'avg_success_eval': 18587.967741935485,
 'ert': 162136.35483870967,
 'failure_rate': 0.7416666666666667,
 'gary_ert': 8294.10396039604,
 'hv': 8114.775000000001,
 'par10': 375635.225,
 'par2': 78968.55833333333,
 'sp': 71953.4235171696,
 'success_cnt': 31,
 'success_rate': 0.25833333333333336,
 'success_rate_length': 0.15491412151601924,
 'success_rate_lower': 0.1883728380805067,
 'success_rate_upper': 0.34328695959652594}
121
5027243


### Convergence plot
![convergence plot for mlsl](../img/mlsl/mlsl-convergence.png)
### Stacked graph (copying last value after termination)
![stacked graph for mlsl (copying last value after termination) ](../img/mlsl/mlsl-stacked-2.png)
### Stacked graph (white after termination)
![stacked graph for mlsl (white after termination)](../img/mlsl/mlsl-stacked-3.png)

## Dual Annealing
### Information and performance measures for the best instance selected

In [15]:
save_handler = SaveHandler('dual-annealing-2')
algo = dual_annealing
algo.load_best_instance(save_handler=save_handler)
ins = algo.best_instance
ins.run(save_handler=save_handler, restart=True, max_instance_fes=MAX_INSTANCE_FES * 5)
pprint(ins.info)
pprint(ins.params)
pprint(ins.performance_measures(excluding_first=True))
pprint(len(ins.results))
pprint(sum([r.eval_num for r in ins.results]))

{'algorithm_name': 'Dual Annealing',
 'algorithm_version': 3,
 'instance_index': 225189433}
{'accept': -3.33337,
 'initial_temp': 79.77754246615842,
 'restart_temp_ratio': 0.0033935664873233104,
 'visit': 2.8292929292929294}
{'avg_height': 1312.102352338263,
 'avg_success_eval': 24189.08,
 'ert': 200189.07999999996,
 'failure_rate': 0.7787610619469026,
 'gary_ert': 10213.728571428572,
 'hv': 5710.380530973452,
 'par10': 394732.09734513273,
 'par2': 83227.67256637168,
 'sp': 109334.6416,
 'success_cnt': 25,
 'success_rate': 0.22123893805309736,
 'success_rate_length': 0.15164100696078606,
 'success_rate_lower': 0.1545837361055079,
 'success_rate_upper': 0.30622474306629394}
114
5004766


### Convergence plot
![convergence plot for da](../img/da/dual-annealing-convergence.png)
### Stacked graph (copying last value after termination)
![stacked graph for da (copying last value after termination) ](../img/da/dual-annealing-stacked-2.png)
### Stacked graph (white after termination)
![stacked graph for da (white after termination)](../img/da/dual-annealing-stacked-3.png)

## CMAES
### Information and performance measures for the best instance selected

In [17]:
save_handler = SaveHandler('cmaes-1')
algo = cmaes
algo.load_best_instance(save_handler=save_handler)
ins = algo.best_instance
ins.run(save_handler=save_handler, restart=True, max_instance_fes=MAX_INSTANCE_FES * 5)
pprint(ins.info)
pprint(ins.params)
pprint(ins.performance_measures(excluding_first=True))
pprint(len(ins.results))
pprint(sum([r.eval_num for r in ins.results]))

{'algorithm_name': 'CMAES', 'algorithm_version': 1, 'instance_index': 64}
{'population_size': 7, 'sigma0_n': 9}
{'avg_height': 889.2217245601929,
 'avg_success_eval': 428.6666666666667,
 'ert': 1666498.666666667,
 'failure_rate': 0.9993772057297073,
 'gary_ert': 2984.7737313432835,
 'hv': 30.87274237077019,
 'par10': 499688.8698359975,
 'par2': 99937.9875441146,
 'sp': 688295.7777777778,
 'success_cnt': 3,
 'success_rate': 0.0006227942702927133,
 'success_rate_length': 0.0016178202387263228,
 'success_rate_lower': 0.00021182451751020114,
 'success_rate_upper': 0.001829644756236524}
4818
5000301


### Convergence plot
![convergence plot for cmaes](../img/cmaes/CMAES-convergence.png)
### Stacked graph (copying last value after termination)
![stacked graph for cmaes (copying last value after termination) ](../img/cmaes/CMAES-stacked-2.png)
### Stacked graph (white after termination)
![stacked graph for cmaes (white after termination)](../img/cmaes/CMAES-stacked-3.png)